## Fitting profiles with Markov Chain Monte Carlo
This is a jupyter notebook intended to fit stellar density profiles to 
a bunch of clusters simulated. It fits the exponential radius, ellipticity, angular position, background density and center shift in ra and dec.

The code runs reading the position of the stars in a catalog and fitting the pars. After the fitting, the results should be compared to the
truth table (the parameters of the clusters simulated).

Open a terminal and run the following commands to create and activate a new environment and install dependencies:
```
conda create -p $HOME/.conda/envs/emcee_env
conda activate emcee_env
conda install -c astropy emcee
conda install -c conda-forge matplotlib
conda install astropy
conda install -c astropy corner
conda install ipykernel
pip install astropy
ipython kernel install --user --name=emcee_env
```
Activate the environment (if you are running the code in Linea's jupyter notebook). Restart the kernel on the jupyter
notebook and load the new environment (clicking on the env on the top right selection).</br>
Ok, now you are ready to run the code. </br>
You can run this code on any machine since you follow the steps above to create an env and install dependencies.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import astropy.io.fits as fits
import corner
import json
import os
from multiprocessing import Pool


Below are a few functions (that will be documented in near future!) to provide the results of the fitting.
This is very similar to the main example available on https://emcee.readthedocs.io/en/stable/tutorials/line/ .
Please, visit that page to have a big picture of the fitting.

In [2]:
def lnprior(theta, thmin, thmax):
    """Calculates if pars are within limits.

    Parameters
    ----------
    theta : list
        Set of parameters.

    Returns
    -------
    float
        Zero or -infinity
    """
    re, ell, th, bg, xc, yc = theta
    if 0.01 < re < 5.0 and 0.0 < ell < 0.6 and thmin < th < thmax and 0.0 < bg < 10. and -3. < xc < 3. and -3. < yc < 3.:
        return 0
    return -np.inf


def lnlike(theta, RA, DEC):
    """Calculates likelihood of a specific set of parameters.

    Parameters
    ----------
    theta : list
        Set of parameters.
    RA : list
        RA of stars, in degrees.
    DEC : list
        DEC of stars, in degrees.

    Returns
    -------
    float
        2*ln likelihood
    """

    re, ell, th, bg, xc, yc = theta
    dX = (RA-(ra0+xc/60.))*np.cos(np.radians(dec0))
    dY = (DEC-(dec0+yc/60.))
    ri = 60.*np.sqrt(((dX*np.cos(np.radians(100*th))-dY*np.sin(np.radians(100*th)))/(1.-ell))
                     ** 2.+(dX*np.sin(np.radians(100*th))+dY*np.cos(np.radians(100*th)))**2.)
    #s0 = (len(RA)-bg*3600.)/((1-ell)*2.*np.pi*re**2)

    #idbg = (ri < 60*R0) & (ri > 60*R1)
    #area_annulus_arcmin = 3600. * np.pi * (R0 ** 2 - R1 ** 2)
    # bg = len(RA[idbg])/ area_annulus_arcmin  # stars/arcmin^2
    # N_star = (len(RA[R < 60.*R0])-(robg * 3600. * np.pi * R0 ** 2))
    s0 = (len(RA)-bg*3600.)/((1-ell)*2.*np.pi*re**2)

    if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
        return -np.inf
    return 2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))


def lnprob(theta, RA, DEC, thmin, thmax):
    """Accounts two previous functions.

    Parameters
    ----------
    theta : list
        set of parameters.
    RA : list
        RA of stars in degrees.
    DEC : list
        DEC of stars in degrees.

    Returns
    -------
    float
        Likelihood. If within the limits it is a number.
    """
    lp = lnprior(theta, thmin, thmax)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, RA, DEC)

In [3]:
# reading config file
with open("fit.json") as f:
    param = json.load(f)
globals().update(param)

os.system('mkdir -p ' + export_dir)

0

In [ ]:
HPX_, ra0_, dec0_, re_kick_, ell_kick_, pa_kick_, dist = np.loadtxt(
    clusters_list_file, usecols=(0, 9, 10, 11, 12, 13, 15), unpack=True)

# Converting exponential radius from parsecs to arcmin
re_kick = 60. * (180. / np.pi) * np.arctan(re_kick_ / dist)

hdu = fits.open(cat_input, memmap=True)
RA_ = hdu[1].data.field('ra')
DEC_ = hdu[1].data.field('dec')
hdu.close()

fff = open(export_dir + 'results.dat', 'w')

print('# HPX re(0,+,-) ell(0,+,-) pa(0,+,-) robg(0,+,-) shift_ra(0,+,-) shift dec(0,+,-)', file=fff)

for i in range(len(ra0_)):
    reinf = 0.5 * re_kick[i]
    resup = 1.5 * re_kick[i]
    thmin = (pa_kick_[i] - 45.) / 100
    thmax = (pa_kick_[i] + 45.) / 100
    
    ra0 = ra0_[i]
    dec0 = dec0_[i]

    cutout_size_RA = R0 / np.cos(np.deg2rad(dec0))

    cond = (RA_ > ra0 - cutout_size_RA) & (RA_ < ra0 +
                                           cutout_size_RA) & (DEC_ > dec0 - R0) & (DEC_ < dec0 + R0)
    RA = RA_[cond]
    DEC = DEC_[cond]

    re_ml = (reinf + resup)/2.
    ell_ml = 0.05
    th_ml = (thmin + thmax) / 2.
    
    R = 60.*np.sqrt(((np.cos(np.radians(dec0))*(ra0-RA))**2.)+(DEC-dec0)**2) # in arcmin

    idbg = (R < 60.*R0) & (R > 60.*R1)
    area_annulus_arcmin = 3600. * np.pi*(R0**2 - R1**2) #  * np.cos(np.radians(dec0))
    robg = len(RA[idbg]) / area_annulus_arcmin  # stars/arcmin^2

    N_star = (len(RA[R < 60.*R0])-(robg*3600.*np.pi * R0 ** 2))
    s_ml = N_star/(2.*np.pi*re_ml**2)
    xc_ml = 0.0
    yc_ml = 0.0
    result = re_ml, ell_ml, th_ml, robg, xc_ml, yc_ml
    print(result)

    ndim, nwalkers = 6, 300  # 300
    pos = [result + 2e-4*np.random.randn(ndim) for j in range(nwalkers)]
    with Pool() as pool:
        sampler = emcee.EnsembleSampler(
            nwalkers, ndim, lnprob, args=(RA, DEC, thmin, thmax), pool=pool)
        sampler.run_mcmc(pos, 300)  # 200
    samples = sampler.chain[:, 150:, :].reshape((-1, ndim))

    samples[:, 2] = 100.*(samples[:, 2])
    re_mcmc, ell_mcmc, th_mcmc, robg_mcmc, xc_mcmc, yc_mcmc = map(lambda v: (
        v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))
    
    N_star_0 = (len(RA)-(robg_mcmc[0] * 3600.))
    N_star_m = N_star_0 - (len(RA)-((robg_mcmc[0] + robg_mcmc[1])* 3600.))
    N_star_p = (len(RA)-((robg_mcmc[0] - robg_mcmc[2])* 3600.)) - N_star_0
    print(int(HPX_[i]), re_mcmc[0], re_mcmc[1], re_mcmc[2], ell_mcmc[0], ell_mcmc[1], ell_mcmc[2],
          th_mcmc[0], th_mcmc[1], th_mcmc[2], robg_mcmc[0], robg_mcmc[1], robg_mcmc[2],
          xc_mcmc[0], xc_mcmc[1], xc_mcmc[2], yc_mcmc[0], yc_mcmc[1], yc_mcmc[2], N_star_0, N_star_p, N_star_m, file=fff)
    
    # Plotting best-fitting pars
    fig = corner.corner(samples, labels=["$re$", "$ell$", "$th$", r"$\rho_{bg}$", r"$\Delta\alpha$", r"$\Delta\delta$"],
                        # truths=[re_mcmc[0], ell_mcmc[0], th_mcmc[0], robg_mcmc[0], xc_mcmc[0], yc_mcmc[0]],
                        truths=[re_kick[i], ell_kick_[i], pa_kick_[i], robg, 0.0000, 0.0000],
                        quantiles=[0.16, 0.5, 0.84], show_titles=True, plot_contours=True)
    value1 = [re_kick[i], ell_kick_[i], pa_kick_[i], 0.0000, 0.0000]
    plt.savefig(export_dir + str(int(HPX_[i])) + '_plus.png')
    plt.close()
    '''
    L = np.zeros(len(RA))
    dX = (RA-(ra0+xc_mcmc[0]/60.))*np.cos(np.radians(dec0))
    dY = (DEC-(dec0+yc_mcmc[0]/60.))
    ri = 60.*np.sqrt(((dX*np.cos(np.radians(100*th_mcmc[0]))-dY*np.sin(np.radians(100*th_mcmc[0])))/(
        1.-ell_mcmc[0]))**2.+(dX*np.sin(np.radians(100*th_mcmc[0]))+dY*np.cos(np.radians(100*th_mcmc[0])))**2.)
    L = (np.exp(-ri/re_mcmc[0]))
    '''
    # Saving chains
    fig1, axes = plt.subplots(6, figsize=(10, 7), sharex=True)
    samples = sampler.get_chain()
    labels = ["rexp", "ell", "theta", "rho_bg", "Ddelta", "Dalpha"]
    for j in range(ndim):
        ax = axes[j]
        ax.plot(samples[:, :, j], "k", alpha=0.3)
        ax.set_xlim(0, len(samples))
        ax.set_ylabel(labels[j])
        ax.yaxis.set_label_coords(-0.1, 0.5)
    axes[-1].set_xlabel("step number")
    plt.savefig(export_dir + str(int(HPX_[i])) + '_chains.png')
    plt.close()
fff.close()

(1.1778902579768156, 0.05, 0.08090000000000003, 0.7213550429952109, 0.0, 0.0)
(1.4219539534370813, 0.05, 1.2699, 0.727249670517133, 0.0, 0.0)
(1.9406029272894756, 0.05, 0.22610000000000002, 0.6963028760270421, 0.0, 0.0)


/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):


(1.556258211755911, 0.05, 0.8308, 0.7386705113408569, 0.0, 0.0)
(2.573212335166782, 0.05, 0.512, 0.7055132315300453, 0.0, 0.0)
(1.559761449171716, 0.05, 0.6408, 0.7150920012531686, 0.0, 0.0)


/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):


(1.576642275563598, 0.05, 1.2851, 0.7430914819822985, 0.0, 0.0)
(2.5035195936425327, 0.05, 1.0475999999999999, 0.7209866287750908, 0.0, 0.0)


/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):


(1.1276567030553688, 0.05, 1.6909, 0.7139867585928082, 0.0, 0.0)
(1.360177646932866, 0.05, 1.1443, 0.7106710306117271, 0.0, 0.0)
(1.42349809215819, 0.05, 0.6193000000000001, 0.7493545237243406, 0.0, 0.0)
(1.5474691761275077, 0.05, 1.0617, 0.7515650090450614, 0.0, 0.0)
(1.9251086677920815, 0.05, 0.9215000000000001, 0.7526702517054218, 0.0, 0.0)
(0.9692162442154639, 0.05, 0.41569999999999996, 0.7320390553786946, 0.0, 0.0)
(0.9413917328770843, 0.05, 1.0674000000000001, 0.7423546535420582, 0.0, 0.0)
(1.1858206479976767, 0.05, 0.6640999999999999, 0.7313022269384544, 0.0, 0.0)
(1.527845067899218, 0.05, 1.4116, 0.7342495406994154, 0.0, 0.0)
(1.0652705552548416, 0.05, 0.4226, 0.7069868884105258, 0.0, 0.0)
(2.6539089456702087, 0.05, 0.23859999999999998, 0.7220918714354512, 0.0, 0.0)
(2.561204092494159, 0.05, 1.5756999999999999, 0.7243023567561719, 0.0, 0.0)
(1.5377418750880119, 0.05, 1.5387, 0.7441967246426588, 0.0, 0.0)
(1.09472780753923, 0.05, 1.3337, 0.7231971140958116, 0.0, 0.0)
(1.57832335

/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
/tmp/ipykernel_26215/3300783791.py:51: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):


In [ ]:
HPX, re_bf, re_bf_p, re_bf_m, ell_bf, ell_bf_p, ell_bf_m, th_bf, th_bf_p, th_bf_m, \
robg_bf, robg_bf_p, robg_bf_m, xc_bf, xc_bf_p, xc_bf_m, yc_bf, yc_bf_p, yc_bf_m, nstar_bf, nstar_bf_p, nstar_bf_m = \
np.loadtxt(export_dir + 'results.dat', unpack=True)
# Truth table : HPX_, ra0_, dec0_, re_kick_, ell_kick_, pa_kick_, dist
HPX0, nstar0, ra0, dec0, re0, ell0, th0, dist0 = np.loadtxt(
    clusters_list_file, usecols=(0, 1, 9, 10, 11, 12, 13, 14), unpack=True)

re0_arcmin = 60. * np.rad2deg(np.arctan(re0 / dist0))


In [ ]:
def identity(min_, max_):
    return np.linspace(min_, max_, 10)

fig, ax = plt.subplots(2, 2, figsize=(18, 18))

ax[0,0].plot(identity(np.min(re0_arcmin), np.max(re0_arcmin)), identity(np.min(re0_arcmin), np.max(re0_arcmin)), lw=1., color='k', ls=':')
ax[0,1].plot(identity(np.min(ell0), np.max(ell0)), identity(np.min(ell0), np.max(ell0)), lw=1., color='k', ls=':')
ax[1,0].plot(identity(-50., 180.), identity(-50., 180.), lw=1., color='k', ls=':')
ax[1,1].plot(identity(50., 180.), identity(50., 180.), lw=1., color='k', ls=':')

ax[0,0].errorbar(re0_arcmin, re_bf, np.array([re_bf_m, re_bf_p]), color='b', fmt='^', label=r'$r_e$', capsize=5)
ax[0,1].errorbar(ell0, ell_bf, np.array([ell_bf_m, ell_bf_p]), color='r', fmt='^', label=r'$ell$', capsize=5)
ax[1,0].errorbar(th0, th_bf, np.array([th_bf_m, th_bf_p]), color='b', fmt='^', label=r'$\theta$', capsize=5)
ax[1,1].errorbar(nstar0, nstar_bf, np.array([nstar_bf_m, nstar_bf_p]), color='b', fmt='^', label=r'$N\ star$', capsize=5)

ax[0,0].set_xlabel(r'$true\ r_e\ (arcmin)$')
ax[0,1].set_xlabel(r'$true\ ellipticity$')
ax[1,0].set_xlabel(r'$true\ position\ angle\ (deg)$')
ax[1,1].set_xlabel(r'$True\ star\ counts$')

ax[0,0].set_ylabel(r'$best-fitting\ r_e\ (arcmin)$')
ax[0,1].set_ylabel(r'$best-fitting\ ellipticity$')
ax[1,0].set_ylabel(r'$best-fitting\ position\ angle\ (deg)$')
ax[1,1].set_ylabel(r'$best-fitting\ star\ counts$')

ax[0,0].set_title(r'$Exponential\ radius\ (arcmin)$')
ax[0,1].set_title(r'$Ellipticity\ \epsilon$')
ax[1,0].set_title(r'$Position\ angle\ (deg)$')
ax[1,1].set_title(r'$Cluster\ stars$')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.grid()
ax.errorbar(xc_bf, yc_bf, xerr=np.array([xc_bf_m, xc_bf_p]), yerr=np.array([yc_bf_m, yc_bf_p]), color='b', lw=1, fmt='b^', capsize=20)
ax.set_title('Position in RA and DEC')
ax.set_xlabel(r'$\Delta\ RA$\ (arcmin) ')
ax.set_ylabel(r'$\Delta\ DEC$\ (arcmin) ')
plt.show()

Plotting each fit trying to see a bias between exponential radius and ellipticity/positional angle.